In [42]:
import pandas as pd
import datetime as dt
import numpy as np

In [65]:
data = pd.read_csv('1_rawdata.csv')
data = data.dropna(how='all')

# Fixing the FIX and minus NoneTime
data.rename(columns={'Unnamed: 7':'fix'}, inplace=True)
for index, dp in data.iterrows():
    if dp['fix'] == 'FIX':
        dp['NoneTime'] = abs(dp['NoneTime'])
        dp['fix'] = ''
data = data.drop(columns=['fix'])

print(data.tail())

       PlaybrushID                             TimestampUTC  UpTime  DownTime  \
1436  PB2500016085  Sat Sep 23 2017 06:01:16 GMT+0100 (BST)    1.25      0.00   
1437  PB2500016085  Sun Sep 24 2017 07:25:38 GMT+0100 (BST)    0.00      0.45   
1438  PB2500016085  Sun Sep 24 2017 07:25:50 GMT+0100 (BST)    1.40      2.75   
1439  PB2500016085  Sun Sep 24 2017 19:01:57 GMT+0100 (BST)    0.05      0.20   
1440  PB2500029963  Fri Sep 22 2017 12:46:30 GMT+0100 (BST)    2.45      0.00   

      LeftTime  RightTime  NoneTime  
1436      0.85       0.00       3.9  
1437      2.75       2.25       3.6  
1438      2.65       0.40       1.8  
1439      0.00       2.55       4.2  
1440      0.15       0.00       8.4  


In [83]:
# convert 'TimestampUTC' into datetime object
data['TimestampUTC'] = pd.to_datetime(data['TimestampUTC'])

# Merge sessions less than 2 minutes apart
for i1, dp1 in data.iterrows():
    print('\r'+str(round(i1/14.4,1))+'%', end='')
    for i2, dp2 in data.iterrows():
        now_time, future_time = dp1['TimestampUTC'], dp2['TimestampUTC']
        two_mins = dt.timedelta(seconds=120)
        if dp1['PlaybrushID'] == dp2['PlaybrushID']:
            if now_time+two_mins > future_time:
                print(dp1['PlaybrushID'])
                data.iloc[-1] = [dp1['PlaybrushID'], dp1['TimestampUTC'], # add a new merged row
                                dp1['UpTime']+dp2['UpTime'],
                                dp1['DownTime']+dp2['DownTime'],
                                dp1['LeftTime']+dp2['LeftTime'],
                                dp1['RightTime']+dp2['RightTime'],
                                dp1['NoneTime']+dp2['NoneTime']
                               ]
                data.drop([i1,i2], axis=0) # delete 2 merged
                
'''two_mins = dt.timedelta(seconds=120)
mlk = np.where((data['datetime'] + two_mins > data['datetime'])
               & (data['PlaybrushID'] == data['PlaybrushID'])
               ,[data['PlaybrushID'], data['datetime']],False)
[print(i) for i in mlk]'''


0.0%PB2500017115
0.1%PB2500017115
PB2500017115
PB2500017115
PB2500017115
0.1%PB2500017115
PB2500017115
PB2500017115
PB2500017115
0.2%PB2500017115
PB2500017115
PB2500017115
PB2500017115
PB2500017115
PB2500017115
0.3%PB2500017115
PB2500017115
PB2500017115
PB2500017115
PB2500017115
PB2500017115
0.3%PB2500017115
PB2500017115
PB2500017115
PB2500017115
PB2500017115
PB2500017115
PB2500017115
PB2500017115
0.4%PB2500017115
PB2500017115
PB2500017115
PB2500017115
PB2500017115
PB2500017115
PB2500017115
PB2500017115
0.5%PB2500017115
PB2500017115
PB2500017115
PB2500017115
PB2500017115
PB2500017115
PB2500017115
PB2500017115
PB2500017115
PB2500017115
0.6%PB2500017115
PB2500017115
PB2500017115
PB2500017115
PB2500017115
PB2500017115
PB2500017115
PB2500017115
PB2500017115
PB2500017115
0.6%PB2500035239
0.7%PB2500016077
0.8%PB2500010629
0.8%PB2500034872
0.9%PB2500034872
PB2500034872
PB2500034872
PB2500034872
PB2500034872
1.0%PB2500034872
PB2500034872
PB2500034872
PB2500034872
PB2500034872
1.0%PB2500034872


KeyboardInterrupt: 

In [ ]:
print(data.tail())

In [ ]:
# Summing the data to get total brushing time.
data['TotTime'] = data['UpTime'] + data['DownTime'] + data['LeftTime'] + data['DownTime'] + data['NoneTime']

# Discard sessions with total time under 20s
